In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import logging
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from PipelineCacheWrapper.PipelineCacheWrapper import PipelineCacheWrapper

In [2]:
from LamaTRExData import LamaTRExData
from SentenceTypologyQueryResults import SentenceTypologyQueryResults
from relation_templates.templates import relations, nominalized_relations

In [3]:
from TypologyQuerier import TypologyQuerier 

In [4]:
logging.set_verbosity_error()

In [5]:
MASK = "[MASK]"
KEYS = ["active", "passive", "nominalized"]

In [6]:
#relations = ["P19", "P36", "P101", "P103","P106","P108", "P178", "P1001"]
#relations = ["P19", "P413", "P159", "P103"]
#relations = ["P364"]
relations = nominalized_relations

In [7]:
TOP_K = 1

In [8]:
model='bert-large-cased'

In [9]:
#unmasker = pipeline('fill-mask', model='bert-large-uncased-whole-word-masking', top_k=100)
unmasker = PipelineCacheWrapper('fill-mask', model=model, top_k=100)

In [10]:
TREx = LamaTRExData(relations = relations)
TREx.load()

In [11]:
querier = TypologyQuerier(unmasker, relations, TOP_K, MASK, KEYS)

In [12]:
querier.query(TREx.data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 949/949 [03:09<00:00,  5.00it/s]


In [13]:
%%time
unmasker.save_to_cache()

CPU times: user 9.37 s, sys: 1.15 s, total: 10.5 s
Wall time: 10.7 s


In [14]:
querier.print_global_result()

total Result
| type        |   correct |   total |   accuracy |
|-------------+-----------+---------+------------|
| active      |       915 |    5093 |    17.9658 |
| passive     |      1487 |    5093 |    29.1969 |
| nominalized |      1015 |    5093 |    19.9293 |
Total Result for 1:1
| type        |   correct |   total |   accuracy |
|-------------+-----------+---------+------------|
| active      |         0 |       0 |          0 |
| passive     |         0 |       0 |          0 |
| nominalized |         0 |       0 |          0 |
Total Result for N:1
| type        |   correct |   total |   accuracy |
|-------------+-----------+---------+------------|
| active      |       705 |    3552 |    19.848  |
| passive     |      1096 |    3552 |    30.8559 |
| nominalized |       890 |    3552 |    25.0563 |
Total Result for N:M
| type        |   correct |   total |   accuracy |
|-------------+-----------+---------+------------|
| active      |       210 |    1541 |   13.6275  |
| pass

In [15]:
querier.print_global_latex()

Total  & 17.97 & 29.20 & 19.93 \\
1:1  & 0.00 & 0.00 & 0.00 \\
N:1  & 19.85 & 30.86 & 25.06 \\
N:M  & 13.63 & 25.37 & 8.11 \\


In [16]:
querier.print_for_latex()

P127 & owned by & N:1 & 9.02 & 34.64 & 30.71 \\
P136 & genre & N:1 & 1.29 & 0.00 & 3.11 \\
P176 & manufacturer & N:1 & 60.29 & 87.37 & 66.19 \\
P413 & position played on team / speciality & N:1 & 4.10 & 0.00 & 0.00 \\
P1303 & instrument & N:M & 8.85 & 0.00 & 0.00 \\
P178 & developer & N:M & 21.28 & 66.05 & 21.11 \\


In [17]:
TOP_K = 10

In [18]:
unmasker = PipelineCacheWrapper('fill-mask', model=model, top_k=100)

In [19]:
querier.reset()
querier.set_top_k(TOP_K)
querier.set_model(unmasker)

In [20]:
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 949/949 [00:00<00:00, 32142.21it/s]


In [21]:
%%time
unmasker.save_to_cache()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs


In [22]:
querier.print_global_result()

total Result
| type        |   correct |   total |   accuracy |
|-------------+-----------+---------+------------|
| active      |      2530 |    5093 |    49.676  |
| passive     |      1916 |    5093 |    37.6203 |
| nominalized |      1873 |    5093 |    36.776  |
Total Result for 1:1
| type        |   correct |   total |   accuracy |
|-------------+-----------+---------+------------|
| active      |         0 |       0 |          0 |
| passive     |         0 |       0 |          0 |
| nominalized |         0 |       0 |          0 |
Total Result for N:1
| type        |   correct |   total |   accuracy |
|-------------+-----------+---------+------------|
| active      |      1764 |    3552 |    49.6622 |
| passive     |      1381 |    3552 |    38.8795 |
| nominalized |      1578 |    3552 |    44.4257 |
Total Result for N:M
| type        |   correct |   total |   accuracy |
|-------------+-----------+---------+------------|
| active      |       766 |    1541 |    49.708  |
| pass

In [23]:
querier.print_global_latex()

Total  & 49.68 & 37.62 & 36.78 \\
1:1  & 0.00 & 0.00 & 0.00 \\
N:1  & 49.66 & 38.88 & 44.43 \\
N:M  & 49.71 & 34.72 & 19.14 \\


In [24]:
querier.print_for_latex()

P127 & owned by & N:1 & 46.00 & 64.19 & 53.28 \\
P136 & genre & N:1 & 22.13 & 0.00 & 43.07 \\
P176 & manufacturer & N:1 & 90.53 & 95.72 & 81.77 \\
P413 & position played on team / speciality & N:1 & 37.08 & 0.00 & 0.84 \\
P1303 & instrument & N:M & 38.78 & 0.11 & 6.32 \\
P178 & developer & N:M & 67.23 & 90.20 & 39.70 \\


In [25]:
TOP_K = 100

In [26]:
unmasker = PipelineCacheWrapper('fill-mask', model=model, top_k=100)

In [27]:
querier.reset()
querier.set_top_k(TOP_K)
querier.set_model(unmasker)

In [28]:
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 949/949 [00:00<00:00, 16142.80it/s]


In [29]:
%%time
unmasker.save_to_cache()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 2.86 µs


In [30]:
querier.print_global_result()

total Result
| type        |   correct |   total |   accuracy |
|-------------+-----------+---------+------------|
| active      |      4068 |    5093 |    79.8743 |
| passive     |      2214 |    5093 |    43.4714 |
| nominalized |      3807 |    5093 |    74.7497 |
Total Result for 1:1
| type        |   correct |   total |   accuracy |
|-------------+-----------+---------+------------|
| active      |         0 |       0 |          0 |
| passive     |         0 |       0 |          0 |
| nominalized |         0 |       0 |          0 |
Total Result for N:1
| type        |   correct |   total |   accuracy |
|-------------+-----------+---------+------------|
| active      |      2994 |    3552 |    84.2905 |
| passive     |      1627 |    3552 |    45.8052 |
| nominalized |      2626 |    3552 |    73.9302 |
Total Result for N:M
| type        |   correct |   total |   accuracy |
|-------------+-----------+---------+------------|
| active      |      1074 |    1541 |    69.695  |
| pass

In [31]:
querier.print_global_latex()

Total  & 79.87 & 43.47 & 74.75 \\
1:1  & 0.00 & 0.00 & 0.00 \\
N:1  & 84.29 & 45.81 & 73.93 \\
N:M  & 69.70 & 38.09 & 76.64 \\


In [32]:
querier.print_for_latex()

P127 & owned by & N:1 & 70.31 & 82.97 & 69.00 \\
P136 & genre & N:1 & 73.68 & 0.97 & 64.66 \\
P176 & manufacturer & N:1 & 96.13 & 98.78 & 92.67 \\
P413 & position played on team / speciality & N:1 & 92.54 & 8.19 & 67.23 \\
P1303 & instrument & N:M & 59.22 & 0.74 & 86.83 \\
P178 & developer & N:M & 86.49 & 97.97 & 60.30 \\
